In [ ]:
# Load ENV variables from .env file
from dotenv import load_dotenv

load_dotenv('.env', override=True)

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain.agents import AgentExecutor
from langchain.utilities import SerpAPIWrapper
from langchain.chat_models.azure_openai import AzureChatOpenAI

In [ ]:
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description
from langchain.memory.buffer import ConversationBufferMemory

In [ ]:
llm = AzureChatOpenAI(
    temperature=0,
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k"
)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world",
    )
]


In [ ]:
prompt = hub.pull("hwchase17/react")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

llm_with_stop = llm.bind(stop=["\nObservation"])

In [ ]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

memory = ConversationBufferMemory(memory_key="chat_history")
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

agent_executor.invoke({"input": "what is the capital of new york?"})["output"]